In [1]:
import pandas as pd
import requests as rq
import re

from bs4 import BeautifulSoup

In [2]:
pd.set_option("display.max_columns", None)

___
## Dados Brutos
Escolhendo as palavras-chave de pesquisa.

In [ ]:
search_words = []

N_words = int(input("Digite a quantidade de palavras-chave que você deseja pesquisar: "))
x = 0
while x < N_words:
    search_word = str(input("Digite a palavra-chave nº" + str((x+1)) + " de pesquisa: "))
    search_words.append(search_word.replace(" ", "+"))
    x = x + 1

N_pages = int(input("Digite a quantidade de páginas do Youtube para se pesquisar: "))

In [ ]:
search_url = "https://www.youtube.com/results?search_query={}&p={}"
search_words

In [ ]:
urls_code = []
urls_searched = []
words_searched = []
for word in search_words:
    for page in range(1, (1+N_pages)):
        url = search_url.format(word, page)
        url_code = rq.get(url)
        words_searched.append(word)
        urls_searched.append(url)
        urls_code.append(url_code.text)

urls_searched = pd.Series(urls_searched)
urls_code = pd.Series(urls_code)

In [ ]:
urls_code.head()

In [ ]:
dados_brutos = pd.DataFrame({"urls": urls_searched, "code": urls_code, "query": words_searched})
dados_brutos.to_csv("dados_brutos.txt", index=False)
dados_brutos.tail()

___
## Processamento dos Dados
Processando os dados para a extração dos campos de informação

In [ ]:
titles = []
urls_videos = []
pages_video = []
queries = []

for n, page in enumerate(dados_brutos["code"].apply(BeautifulSoup)):
    for link in page.findAll("a"):
        if link.has_attr("aria-describedby") == True:
            queries.append(dados_brutos["query"].iloc[n])
            titles.append(link["title"])
            urlls_videos = "https://youtube.com.br" + link["href"]
            urls_videos.append(urlls_videos)
            pages_video.append(rq.get(urlls_videos).text)

In [ ]:
data = pd.DataFrame({"title": titles, "urls_videos": urls_videos, "query": queries, "pages": pages_video})

In [ ]:
data.head()

In [ ]:
data.to_csv("dados_processados.csv", index=False)

___
## Coletando dados dos vídeos
Processo de extração dos campos de informações das páginas de cada vídeo

In [ ]:
data.tail()

In [ ]:
gots_datas = []
for page in data["pages"]:
    parsed = BeautifulSoup(page, "html.parser")
    
    class_watch = parsed.find_all(attrs={"class":re.compile(r"watch")})
    id_watch = parsed.find_all(attrs={"id":re.compile(r"watch")})
    channel = parsed.find_all("a", attrs={"href":re.compile(r"channel")})
    meta = parsed.find_all("meta")
    
    got_data = {}
    
    for cl in class_watch:
        colname = "_".join(cl["class"])
        if "clearfix" in colname:
            continue
        got_data[colname] = cl.text.strip()
    
    for cl in id_watch:
        colname = cl["id"]
        got_data[colname] = cl.text.strip()
        
    for cl in meta:
        colname = cl.get("property")
        if colname is not None:
            got_data[colname] = cl["content"]
    
    for n, cl in enumerate(channel):
        got_data["channel_link_{}".format(n)] = cl["href"]
    
    gots_datas.append(got_data)

In [ ]:
adapt_dict= {}
for colname in gots_datas[0].keys():
    adapt_dict[colname] = []

In [ ]:
all_dfs = list(pd.Series(gots_datas).apply(lambda x: pd.DataFrame(x, index=range(1))))

In [ ]:
videos_data = pd.concat(all_dfs).reset_index(drop=True)

In [ ]:
videos_data.to_csv("dados_dos_videos.csv")

___
## Verificação
Verificação da extração correta dos dados e geração de uma base de informações limpa e adequada

In [ ]:
videos_data.shape

In [ ]:
videos_data.head()

In [ ]:
selected_columns = ["watch-title", "watch-view-count", "watch-time-text", "content_watch-info-tag-list", "watch7-headline",
                   "watch7-user-header", "watch8-sentiment-actions", "og:image", "og:image:width", "og:image:height",
                   "og:description", "og:video:width", "og:video:height", "og:video:type", "channel_link_0"]

In [ ]:
videos_data[selected_columns].to_csv("features_selecionadas(sem labels).csv")

In [ ]:
videos_data[selected_columns].head(10)

___
## Preenchimento dos labels de interesse
Após a etapa anterior, os labels de classficiação dos vídeos, caso seja do meu interesse ou não assistí-los, será preenchido manualmente através de uma planilha (Excel) no campo "interested", sendo marcados como: 

- **1:** tenho interesse em assistir aquele vídeo
- **0:** não tenho interesse em assistir aquele vídeo